# 台指選擇權put_call_ratio值抓取  
* 透過openapi取得  
> [臺指選擇權Put/Call比](https://openapi.taifex.com.tw/#/%E8%B3%87%E6%96%99%E6%9F%A5%E8%A9%A2API/get_PutCallRatio)
* 舊數據透過網頁查詢下載csv檔案, 並將csv檔案轉存至duckdb  
> 台灣期交所首頁 > 交易資訊 > 交易資訊 選擇權 > 臺指選擇權Put/Call比  
> [臺指選擇權(TXO)Put/Call Ratios](https://www.taifex.com.tw/cht/3/pcRatio)

In [1]:
import os
import sys
from pathlib import Path
from datetime import date
from datetime import datetime
import requests
import pandas as pd
import duckdb

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.common.duckdb_tool import insert_dataframe_to_duckdb

## 公用參數設定

In [3]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [4]:
# 新增偽裝成chrome瀏覽器的標頭
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

## 外部資料讀取  
### 方法1: 透過期交所 open api 取得最近一個月數據  
* API無法取得當天最新數據，只能取得Ｄ-1交易日資訊

In [ ]:
# 取得散戶小台淨未平倉口數資料
taifex_put_call_ratio = requests.get(
    "https://openapi.taifex.com.tw/v1/PutCallRatio", 
    verify=False,
    headers=headers
).json()
print(taifex_put_call_ratio)

In [ ]:
# json 轉成 dataframe
taifex_put_call_ratio_df = pd.DataFrame(taifex_put_call_ratio)
print(taifex_put_call_ratio_df.shape)
taifex_put_call_ratio_df

### 方法2: 透過期交所網站下載資料進行更新  
* 這方法可以取得最新交易日當天的資訊  

In [ ]:
# 設置請求頭,模擬瀏覽器行為
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.taifex.com.tw/cht/3/pcRatio'
}

# 日期參數設定
# input_date = "2024/08/05"  # 保留作為手動指定日期
input_date = date.today().strftime("%Y/%m/%d")  # 預設指定當天日期

end_date = pd.to_datetime(input_date)
last_month = (end_date - pd.DateOffset(days=30)).strftime("%Y/%m/%d")

# 設置請求參數
# payload = {
#     'down_type': '',
#     'queryStartDate': '2024/09/04',
#     'queryEndDate': '2024/10/04'
# }
payload = {
    'down_type': '',
    'queryStartDate': last_month,
    'queryEndDate': end_date.strftime("%Y/%m/%d")
}
print(payload)

# 發送POST請求
url = 'https://www.taifex.com.tw/cht/3/pcRatioDown'
response = requests.post(url, headers=headers, data=payload)

# 檢查請求是否成功
if response.status_code == 200:
    # 將內容保存為CSV檔案
    with open('tw_pc_ratio.csv', 'wb') as file:
        file.write(response.content)
    print("檔案已成功下載為 tw_pc_ratio.csv")

    # 讀取CSV檔案並顯示內容，設定no header
    tw_pc_ratio_df = pd.read_csv('tw_pc_ratio.csv', skiprows=1, header=None, encoding='big5')
    
    # tw_pc_ratio_df移除最後一欄都是空的資料
    tw_pc_ratio_df = tw_pc_ratio_df.dropna(how='all', axis=1)
    tw_pc_ratio_df.columns = taifex_put_call_ratio_df.columns
    tw_pc_ratio_df.loc[:,"Date"] = pd.to_datetime(tw_pc_ratio_df["Date"])

    os.remove('tw_pc_ratio.csv')
else:
    print(f"請求失敗,狀態碼: {response.status_code}")


In [ ]:
# 將tw_pc_ratio_df 複製給 taifex_put_call_ratio_df
taifex_put_call_ratio_df = tw_pc_ratio_df.copy()

print(taifex_put_call_ratio_df.shape)
taifex_put_call_ratio_df.head(3)

## 資料留存ＤＢ

In [9]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [10]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [11]:
table_name = "tw_pc_ratio"

In [ ]:
insert_row_count = insert_dataframe_to_duckdb(
    conn_duckdb, 
    taifex_put_call_ratio_df, 
    table_name, 
    date_column='Date'
)

f"成功插入 {insert_row_count} 筆資料到 {table_name}"

In [ ]:
# 查詢表中所有資料
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc").fetch_df()

In [14]:
# 關閉資料庫連線
conn_duckdb.close()